In [ ]:
#pip install snowflake-connector-python
#pip install snowflake-sqlalchemy

In [1]:
import pandas as pd
import numpy as np
import json
import requests
from io import StringIO
from google.cloud import storage
from math import ceil
import datetime
import calendar
from sqlalchemy import create_engine
from snowflake.connector import connect
from google.cloud import storage
from snowflake.connector import connect
from sqlalchemy import create_engine
from sqlalchemy.dialects import registry


In [3]:
# Connect to snowflake 
# Read credentials from config file
with open('config_dw.json', 'r') as f:
    config = json.load(f)

snowflake_config = config['snowflake']

# Your Snowflake database credentials
account_name = snowflake_config['account_name']
user = snowflake_config['user']
password = snowflake_config['password']
database = snowflake_config['database']
schema = snowflake_config['schema']

# Establish a connection to Snowflake
conn = connect(
    user=user,
    password=password,
    account=account_name,
    database=database,
    schema=schema
)

registry.register('snowflake', 'snowflake.sqlalchemy', 'dialect')

# Create a SQLAlchemy engine for Snowflake
snowflake_engine = create_engine(f'snowflake://{user}:{password}@{account_name}/{database}/{schema}')
connection = snowflake_engine.connect()
results = connection.execute('select current_version()').fetchone()
print(results[0])

8.15.0


In [3]:
!curl ipecho.net/plain

207.38.251.159

In [4]:
# Your configuration data
import json
from google.cloud import storage

# Load configuration from file
def load_config(filename):
    with open(filename, 'r') as file:
        config_data = json.load(file)
    return config_data

# Load configuration from file
config = load_config('config.json')

# Authenticate with Google Cloud Storage using the service account JSON
storage_client = storage.Client.from_service_account_info(config)

# Google Cloud Storage Configuration
BUCKET_NAME = "housingproject_cis9440"

# Get the bucket
bucket = storage_client.get_bucket(BUCKET_NAME)

# List all blobs in the specified container
blob_list = bucket.list_blobs()

# Iterate over each blob
for blob in blob_list:
    blob_name = blob.name
    print(f"Processing blob: {blob_name}")

    # Check if the blob is a file (not a directory)
    if '.' in blob_name:
        # Get the blob
        blob = bucket.blob(blob_name)

        # Download the blob content
        blob_content = blob.download_as_string()

        # Convert blob content to DataFrame
        df = pd.read_csv(StringIO(blob_content.decode('utf-8')))

        # Display the shape of the DataFrame
        print(f"Shape of {blob_name}: {df.shape}")
    else:
        print("Blob is a directory, skipping...")


Processing blob: 0.json
Shape of 0.json: (0, 43429)
Processing blob: MIT_livingwage_calculation.csv
Shape of MIT_livingwage_calculation.csv: (186, 14)
Processing blob: MIT_typicalannualsalaries.csv
Shape of MIT_typicalannualsalaries.csv: (1364, 3)
Processing blob: MIT_typicalexpenses.csv
Shape of MIT_typicalexpenses.csv: (682, 14)
Processing blob: NYS crime data.csv
Shape of NYS crime data.csv: (75, 13)
Processing blob: SALES/NY/0.json
Shape of SALES/NY/0.json: (0, 43429)
Processing blob: SALES/NY/1000.json
Shape of SALES/NY/1000.json: (0, 43237)
Processing blob: SALES/NY/10000.json
Shape of SALES/NY/10000.json: (0, 43298)
Processing blob: SALES/NY/100000.json
Shape of SALES/NY/100000.json: (0, 43487)
Processing blob: SALES/NY/101000.json
Shape of SALES/NY/101000.json: (0, 43728)
Processing blob: SALES/NY/102000.json
Shape of SALES/NY/102000.json: (0, 44572)
Processing blob: SALES/NY/103000.json
Shape of SALES/NY/103000.json: (0, 43386)
Processing blob: SALES/NY/104000.json
Shape of SA

In [5]:
# Recreate the iterator blob_list
blob_list = bucket.list_blobs()

# Iterate over each blob
for blob in blob_list:
    blob_name = blob.name
    print(f"Processing blob: {blob_name}")

    #CHANGE THIS CODE TO UPDATE ONLT THE NEEDED FILES (DO NOT UPDATE ALL AGAIN SINCE WE ALREADY HAVE SOME IN SNOWFALKE)
    # Check if the blob is a file (not a directory) 
    if '.' in blob_name and 'SCHOOL' in blob_name:
        # Get the blob
        blob = bucket.blob(blob_name)

        # Download the blob content
        blob_content = blob.download_as_string()

        # Convert blob content to DataFrame
        df = pd.read_csv(StringIO(blob_content.decode('utf-8')))

        # Display the shape of the DataFrame
        print(f"Shape of {blob_name}: {df.shape}")

        # Store the DataFrame into Snowflake
        table_name = blob_name.split('.')[0]  # Extract table name from blob name
        df.to_sql(table_name, con=snowflake_engine, if_exists='append', index=False)
        print(f"DataFrame stored in Snowflake table: {table_name}")
    else:
        print("Blob is not a file, skipping...")

Processing blob: 0.json
Blob is not a file, skipping...
Processing blob: MIT_livingwage_calculation.csv
Blob is not a file, skipping...
Processing blob: MIT_typicalannualsalaries.csv
Blob is not a file, skipping...
Processing blob: MIT_typicalexpenses.csv
Blob is not a file, skipping...
Processing blob: NYS crime data.csv
Blob is not a file, skipping...
Processing blob: SALES/NY/0.json
Blob is not a file, skipping...
Processing blob: SALES/NY/1000.json
Blob is not a file, skipping...
Processing blob: SALES/NY/10000.json
Blob is not a file, skipping...
Processing blob: SALES/NY/100000.json
Blob is not a file, skipping...
Processing blob: SALES/NY/101000.json
Blob is not a file, skipping...
Processing blob: SALES/NY/102000.json
Blob is not a file, skipping...
Processing blob: SALES/NY/103000.json
Blob is not a file, skipping...
Processing blob: SALES/NY/104000.json
Blob is not a file, skipping...
Processing blob: SALES/NY/105000.json
Blob is not a file, skipping...
Processing blob: SALES

/var/folders/y5/y4jkcjwx1zsdr2dqg2zg998r0000gn/T/ipykernel_1827/1925320818.py:25: UserWarning: The provided table name 'SCHOOL_DATA' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  df.to_sql(table_name, con=snowflake_engine, if_exists='append', index=False)


DataFrame stored in Snowflake table: SCHOOL_DATA
